In [ ]:
import os
os.chdir("..")

In [ ]:
from src.data.data_process import DataReg
import polars as pl
import requests
import geopandas as gpd
from shapely import wkt
dr = DataReg()

In [ ]:
dr.pull_dp03()

In [ ]:
gdf = gpd.GeoDataFrame(dr.make_spatial_table().df())
gdf["geometry"] = gdf["geometry"].apply(wkt.loads)
gdf = gdf.set_geometry("geometry")
gdf["zipcode"] = gdf["zipcode"].astype(str)
gdf.plot()


In [ ]:
gdf = gpd.read_file(f"{dr.saving_dir}external/zips_shape.zip")
gdf = gdf[gdf["ZCTA5CE20"].str.startswith("00")]
gdf = gdf.rename(columns={"ZCTA5CE20": "zipcode"}).reset_index()
gdf

In [ ]:
_year = 2012
tmp = dr.pull_query(
    params=[
        "DP03_0051E",
        "DP03_0052E",
        "DP03_0053E",
        "DP03_0054E",
        "DP03_0055E",
        "DP03_0056E",
        "DP03_0057E",
        "DP03_0058E",
        "DP03_0059E",
        "DP03_0060E",
        "DP03_0061E",
    ],
    year=_year,
)
tmp = tmp.rename(
    {
        "dp03_0051e": "total_house",
        "dp03_0052e": "inc_less_10k",
        "dp03_0053e": "inc_10k_15k",
        "dp03_0054e": "inc_15k_25k",
        "dp03_0055e": "inc_25k_35k",
        "dp03_0056e": "inc_35k_50k",
        "dp03_0057e": "inc_50k_75k",
        "dp03_0058e": "inc_75k_100k",
        "dp03_0059e": "inc_100k_150k",
        "dp03_0060e": "inc_150k_200k",
        "dp03_0061e": "inc_more_200k",
    }
)
tmp = tmp.rename({"zip code tabulation area": "zipcode"})
tmp.

In [ ]:
params=[
    "DP03_0051E",
    "DP03_0052E",
    "DP03_0053E",
    "DP03_0054E",
    "DP03_0055E",
    "DP03_0056E",
    "DP03_0057E",
    "DP03_0058E",
    "DP03_0059E",
    "DP03_0060E",
    "DP03_0061E",
]
year = 2012
param = ",".join(params)
base = "https://api.census.gov/data/"
flow = "/acs/acs5/profile"
url = f"{base}{year}{flow}?get={param}&for=zip%20code%20tabulation%20area:*&in=state:72"
df = pl.DataFrame(requests.get(url).json())

# get names from DataFrame
names = df.select(pl.col("column_0")).transpose()
names = names.to_dicts().pop()
names = dict((k, v.lower()) for k, v in names.items())

# Pivot table
df = df.drop("column_0").transpose()
df.rename(names).with_columns(year=pl.lit(year))